<a href="https://colab.research.google.com/github/futartup/S4-EVA-Code/blob/master/S4_assignment_anup_gogoi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # self.conv1 = nn.Conv2d(1, 10, 3, padding=1, bias=False)
        # self.conv2 = nn.Conv2d(10, 16, 3, padding=1, bias=False)
        # self.pool1 = nn.MaxPool2d(2, 2)
        # self.dout1 = nn.Dropout(p=0.25)
        # self.conv3 = nn.Conv2d(16, 32, 3, padding=1, bias=False)
        # self.conv4 = nn.Conv2d(32, 64, 3, padding=1, bias=False)
        # self.pool2 = nn.MaxPool2d(2, 2)
        # self.dout2 = nn.Dropout(p=0.25)
        # self.conv5 = nn.Conv2d(64, 64, 1, padding=1, bias=False)
        # self.gap   = nn.AvgPool2d(7)

        # self.conv7 = nn.Conv2d(64, 10, 1)
        #self.conv6 = nn.Conv2d(64, 128, 3, padding=1, bias=False)
        #self.conv7 = nn.Conv2d(128, 10, 3, padding=1, bias=False)

        # self.conv1 = nn.Conv2d(1, 8, 3, padding=1, bias=False)
        # self.pool1 = nn.MaxPool2d(2,2)

        # self.conv2 = nn.Conv2d(8, 16, 3, padding=1, bias=False)
        # self.pool2 = nn.MaxPool2d(2,2)

        # self.conv3 = nn.Conv2d(16, 32, 3, padding=1, bias=False)
        # self.conv4 = nn.Conv2d(32, 64, 1, padding=1, bias=False)
        # self.gap = nn.AvgPool2d(7)

        # self.conv5 = nn.Conv2d(64, 10, 1, padding=1, bias=False)

        '''
        Transition layers (Convolution + pooling) = 3 
        Position of Transition layers = last of each block
        '''
        self.conv1 = nn.Sequential(
            # BLOCK 1
            # we are using 3 * 3 convolution
            nn.Conv2d(1, 8, 3, padding=1, bias=False),
            nn.ReLU(),
            # using batch normalization
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 8, 3, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 8, 3, stride=1, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.MaxPool2d(2, 2),
            # Dropping out one fourth quarter of pixels since the image is of very small size
            nn.Dropout(0.25),

            # BLOCK 2
            nn.Conv2d(8, 16, 3, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3, stride=1, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),

            # BLOCK 3 
            nn.Conv2d(16, 64, 3, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),
            # here we are using 1 * 1 convolution
            nn.Conv2d(64, 10, 1, padding=1, bias=False),

            # OUTPUT
            nn.AvgPool2d(3)
        )
        
    def forward(self, x):
        # x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        # x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        # x = self.gap(F.relu(self.conv5(x)))
        # x = self.conv7(x)

        # x = self.pool1(F.relu(self.conv1(x)))
        # x = self.pool2(F.relu(self.conv2(x)))
        # x = self.gap(F.relu(self.conv4(F.relu(self.conv3(x)))))
        # x = self.conv5(x)
        # x = x.view(-1, 10)
        # return F.log_softmax(x)

        x = self.conv1(x)
        #x = self.conv3(x)
        
        
        #x = self.fc(x)
        x = x.view(-1, 10)
        x = F.log_softmax(x, dim=1)
        return x

In [0]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
              ReLU-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
            Conv2d-4            [-1, 8, 28, 28]             576
              ReLU-5            [-1, 8, 28, 28]               0
       BatchNorm2d-6            [-1, 8, 28, 28]              16
            Conv2d-7            [-1, 8, 28, 28]             576
              ReLU-8            [-1, 8, 28, 28]               0
       BatchNorm2d-9            [-1, 8, 28, 28]              16
        MaxPool2d-10            [-1, 8, 14, 14]               0
          Dropout-11            [-1, 8, 14, 14]               0
           Conv2d-12           [-1, 16, 14, 14]           1,152
             ReLU-13           [-1, 16, 14, 14]               0
      BatchNorm2d-14           [-1, 16,

In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# Here we are using image Normalizer
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 19):
    print("epoch no: " , epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

epoch no:  1


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0829, Accuracy: 9757/10000 (97.6%)

epoch no:  2


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0207, Accuracy: 9935/10000 (99.3%)

epoch no:  17


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0198, Accuracy: 9940/10000 (99.4%)

epoch no:  18


100%|██████████| 469/469 [00:10<00:00, 46.84it/s]



Test set: Average loss: 0.0197, Accuracy: 9936/10000 (99.4%)

